# Cooper 142 SNPs set


## Preparation

### Import required packages.

In [1]:
import os, sys, warnings
import numpy as np
import pandas as pd
import statistics as st
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.exceptions import ConvergenceWarning

### Read input matrix with genotypes

The matrix contains the genotypes from AMP-PD/MGRB dataset for 140 SNPs.

In [2]:
table = pd.read_csv("data/matrix.txt", sep="\t")
table

,participant_id,phenotype,cohort,gender,inv_genotype,rs2275579,rs144115304,rs115581042,rs79531911,rs138844738,...,rs10448130,rs34288580,rs34992950,rs7387252,rs2410595,rs41311559,rs148894916,rs112957100,rs143756122,rs148514732
0,SY-NIH_INVAA791MKCET,1,STEADY-PD3,M,NI,0,0,0,0,0,...,2,2,1,2,1,0,0,0,0,0
1,SY-NIH_INVEP886EEYYL,1,STEADY-PD3,M,NI,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,SY-NIH_INVFM717GWDX4,1,STEADY-PD3,F,NI,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,SY-NIH_INVNN611MKKN9,1,STEADY-PD3,M,NI,0,0,0,0,0,...,1,0,1,1,0,0,0,0,0,0
4,SY-NIH_INVRB171EXGUK,1,STEADY-PD3,M,II,0,1,1,1,0,...,1,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3107,BABQX,0,MGRB,M,II,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3108,BABRB,0,MGRB,F,II,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3109,BABRE,0,MGRB,M,NI,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3110,ZAAAB,0,MGRB,M,NI,0,0,0,0,0,...,2,1,1,1,0,0,0,0,0,0


### Distribution of data

#### Distribution by phenotype

(0=Control, 1=Case)

In [3]:
table.groupby('phenotype')['participant_id'].nunique()

phenotype
0    1556
1    1556
Name: participant_id, dtype: int64

#### Distribution by gender/phenotype

In [4]:
table.groupby(['gender', 'phenotype'])['participant_id'].nunique()

gender  phenotype
F       0            567
        1            567
M       0            989
        1            989
Name: participant_id, dtype: int64

#### Distribution by gender/phenotype/inv8_001 genotype

In [5]:
table.groupby(['gender', 'phenotype', 'inv_genotype'])['participant_id'].nunique()

gender  phenotype  inv_genotype
F       0          II              195
                   NI              259
                   NN              113
        1          II              175
                   NI              270
                   NN              122
M       0          II              318
                   NI              480
                   NN              191
        1          II              296
                   NI              477
                   NN              216
Name: participant_id, dtype: int64

## All participants

### Logistic regression model

In [6]:
pd.set_option('display.max_rows', 150)
X = table[table.columns[5:]]
Y = table['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table.groupby('phenotype')['participant_id'].nunique()

phenotype
0    1556
1    1556
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [7]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#              'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#              'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.01, 0.02, 0.05],
              'max_iter': [10, 25, 50],
              'l1_ratio': [1, 0.9, 0.8]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.01, 0.02, 0.05], 'l1_ratio': [1, 0.9, 0.8],
                         'max_iter': [10, 25, 50]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [8]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.5551641873897212

Non-zero coefficients: 2

Best estimator: LogisticRegression(C=0.02, l1_ratio=1, max_iter=25, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.02, 'l1_ratio': 1, 'max_iter': 25}
Best score: 0.5492252417764205



,SNP,Coefficient
Index,,
1,rs11248057,0.055163
2,rs3806760,0.098122


### 10-fold cross validation (5-times)

In [9]:
name = "all"
AUCs = list()
bp = grid_lr.best_params_
rkf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=42)
clf = LogisticRegression(C=bp['C'], max_iter=bp['max_iter'], l1_ratio=bp['l1_ratio'], random_state=42,
      solver='saga', n_jobs=-1, penalty='elasticnet')

count = 0
for train_index, test_index in rkf.split(X):
    count = count + 1
    print(count, "TRAIN:", train_index, "TEST:", test_index)

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    clf.fit(X_train, y_train)

    y_pred = clf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_pred)
    AUCs.append(auc)

    X_header = np.array(X_train.columns)
    data_array = np.vstack((X_header, clf.coef_[0,:]))
    model_weights = pd.DataFrame(data=data_array.T,columns=['SNP', 'Coefficient'])
    m_name = f'data/{name}_10fold_repeat{count}_coefficients.txt'
    model_weights.to_csv(m_name, sep='\t',index=False, line_terminator='\n')

# Fit predictor to statistically significant features (just once!!!)
clf.fit(X, Y)
y_pred = clf.predict_proba(X)[:,1]

# This in-sample AUC should be better than the AUCs from the repeated cross-validation
auc = roc_auc_score(Y, y_pred)

#AUC results from the 50 predictors
AUC_out = pd.DataFrame(AUCs, columns=['AUC'])
AUC_out.to_csv(f"data/{name}_AUCs.txt", sep='\t',index=False, line_terminator='\n')

AUC_std= st.stdev(AUCs)
AUC_mean= st.mean(AUCs)

num_coef = np.sum(clf.coef_[0,:] != 0)
print(f'In-Sample AUC: {auc}')
print(f'MeanCV AUC: {AUC_mean}')
print(f'Standard Deviation CV AUC: {AUC_std}')
print(f'Non-zero coefficients: {num_coef}')

1 TRAIN: [   1    2    3 ... 3108 3109 3110] TEST: [   0   14   17   25   31   32   44   45   51   52   63   67   70   76
   88   93  102  120  134  139  144  149  152  170  174  175  178  184
  188  192  194  196  203  211  214  218  240  246  251  254  256  257
  266  270  289  291  298  299  309  314  321  322  331  332  343  346
  354  370  408  410  411  416  423  430  433  438  439  443  450  486
  495  506  507  528  535  544  554  555  557  564  565  567  568  572
  581  599  602  605  610  611  612  679  680  685  695  700  705  755
  759  765  772  789  790  794  798  805  840  857  862  867  887  897
  900  903  929  930  932  937  945  949  976  978 1018 1023 1027 1044
 1053 1064 1068 1073 1084 1091 1093 1102 1106 1116 1117 1149 1177 1179
 1206 1207 1210 1211 1216 1221 1222 1242 1251 1268 1307 1309 1317 1320
 1337 1344 1361 1375 1376 1422 1425 1448 1450 1476 1480 1483 1494 1497
 1506 1557 1563 1566 1567 1583 1590 1608 1616 1628 1665 1690 1702 1706
 1716 1718 1721 1723 1742 

/var/folders/y0/1x_8r5js5_z6xvnv37zjwr6r0000gn/T/ipykernel_59613/1802922695.py:25: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  model_weights.to_csv(m_name, sep='\t',index=False, line_terminator='\n')
/var/folders/y0/1x_8r5js5_z6xvnv37zjwr6r0000gn/T/ipykernel_59613/1802922695.py:25: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  model_weights.to_csv(m_name, sep='\t',index=False, line_terminator='\n')
/var/folders/y0/1x_8r5js5_z6xvnv37zjwr6r0000gn/T/ipykernel_59613/1802922695.py:25: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  model_weights.to_csv(m_name, sep='\t',index=False, line_terminator='\n')
/var/folders/y0/1x_8r5js5_z6xvnv37zjwr6r0000gn/T/ipykernel_59613/1802922695.py:25: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  model_weights.to_csv(m_name, sep='\t',index=False, line_terminator='\n')


5 TRAIN: [   0    1    2 ... 3108 3109 3111] TEST: [   8   13   20   23   71   78   83   91   99  105  115  121  132  142
  151  156  158  185  191  198  205  207  209  213  222  273  277  287
  294  305  306  308  311  316  352  374  377  380  382  386  394  409
  421  425  427  429  444  447  458  483  491  500  514  519  526  529
  534  548  552  556  570  571  583  585  588  590  591  607  636  672
  674  686  694  715  731  733  749  751  754  778  792  800  808  836
  855  861  866  873  883  885  886  888  898  902  906  908  910  915
  917  923  931  947  970  973  988  993 1004 1007 1050 1054 1056 1083
 1087 1096 1098 1185 1186 1189 1230 1245 1247 1259 1260 1265 1273 1281
 1293 1328 1338 1343 1349 1351 1362 1368 1397 1399 1405 1410 1417 1435
 1453 1457 1461 1462 1475 1487 1488 1502 1505 1525 1532 1543 1547 1551
 1556 1559 1569 1613 1614 1624 1626 1644 1655 1656 1659 1662 1675 1694
 1703 1710 1727 1735 1744 1764 1767 1769 1800 1805 1807 1832 1844 1868
 1882 1883 1885 1891 1897 

/var/folders/y0/1x_8r5js5_z6xvnv37zjwr6r0000gn/T/ipykernel_59613/1802922695.py:25: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  model_weights.to_csv(m_name, sep='\t',index=False, line_terminator='\n')
/var/folders/y0/1x_8r5js5_z6xvnv37zjwr6r0000gn/T/ipykernel_59613/1802922695.py:25: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  model_weights.to_csv(m_name, sep='\t',index=False, line_terminator='\n')
/var/folders/y0/1x_8r5js5_z6xvnv37zjwr6r0000gn/T/ipykernel_59613/1802922695.py:25: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  model_weights.to_csv(m_name, sep='\t',index=False, line_terminator='\n')
/var/folders/y0/1x_8r5js5_z6xvnv37zjwr6r0000gn/T/ipykernel_59613/1802922695.py:25: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  model_weights.to_csv(m_name, sep='\t',index=False, line_terminator='\n')


9 TRAIN: [   0    1    2 ... 3109 3110 3111] TEST: [   4   11   16   19   35   36   38   46   50   75   89   90  114  116
  119  127  133  143  153  154  172  180  190  216  225  234  245  264
  268  284  301  335  356  369  395  396  399  412  417  469  487  496
  499  504  510  512  515  524  537  546  550  559  569  574  580  595
  606  616  625  633  635  639  652  656  658  675  681  684  699  703
  713  717  728  732  740  750  753  773  784  797  823  825  828  830
  833  837  850  851  853  872  876  884  894  895  896  920  928  933
  953  954  957  959  971  977  980  991  992  996 1008 1012 1015 1045
 1060 1062 1065 1066 1069 1077 1081 1092 1115 1119 1122 1126 1139 1140
 1141 1148 1150 1155 1156 1166 1167 1191 1197 1205 1209 1212 1214 1217
 1219 1227 1243 1296 1312 1324 1346 1348 1365 1384 1388 1409 1416 1440
 1470 1492 1504 1516 1519 1531 1540 1542 1548 1568 1571 1595 1625 1630
 1635 1649 1664 1676 1679 1680 1683 1686 1687 1693 1705 1715 1758 1773
 1781 1782 1785 1799 1810 

...

/var/folders/y0/1x_8r5js5_z6xvnv37zjwr6r0000gn/T/ipykernel_59613/1802922695.py:25: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  model_weights.to_csv(m_name, sep='\t',index=False, line_terminator='\n')
/var/folders/y0/1x_8r5js5_z6xvnv37zjwr6r0000gn/T/ipykernel_59613/1802922695.py:25: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  model_weights.to_csv(m_name, sep='\t',index=False, line_terminator='\n')
/var/folders/y0/1x_8r5js5_z6xvnv37zjwr6r0000gn/T/ipykernel_59613/1802922695.py:25: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  model_weights.to_csv(m_name, sep='\t',index=False, line_terminator='\n')
/var/folders/y0/1x_8r5js5_z6xvnv37zjwr6r0000gn/T/ipykernel_59613/1802922695.py:25: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  model_weights.to_csv(m_name, sep='\t',index=False, line_terminator='\n')


45 TRAIN: [   0    1    2 ... 3108 3109 3111] TEST: [   4    5    8   14   25   32   39   50   67   68   86   90   98  108
  112  119  168  183  185  190  192  203  219  223  243  255  266  269
  285  286  296  316  350  353  358  366  389  421  422  429  431  434
  445  461  477  480  487  491  499  507  509  525  536  549  553  561
  564  565  569  587  595  597  598  609  619  645  652  669  672  680
  681  682  694  699  719  743  744  760  778  792  803  810  821  827
  835  836  839  842  844  874  879  897  903  916  930  931  932  936
  938  979  998 1004 1005 1011 1015 1022 1023 1046 1061 1066 1078 1082
 1086 1101 1115 1125 1145 1157 1159 1172 1176 1177 1185 1190 1236 1249
 1263 1268 1270 1282 1330 1337 1362 1366 1367 1369 1371 1387 1402 1404
 1417 1450 1477 1486 1495 1496 1502 1514 1515 1522 1530 1560 1569 1587
 1632 1633 1638 1649 1659 1664 1665 1674 1678 1688 1693 1695 1705 1719
 1727 1729 1731 1748 1751 1768 1784 1785 1788 1797 1799 1820 1823 1829
 1831 1833 1836 1838 1847

/var/folders/y0/1x_8r5js5_z6xvnv37zjwr6r0000gn/T/ipykernel_59613/1802922695.py:25: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  model_weights.to_csv(m_name, sep='\t',index=False, line_terminator='\n')
/var/folders/y0/1x_8r5js5_z6xvnv37zjwr6r0000gn/T/ipykernel_59613/1802922695.py:25: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  model_weights.to_csv(m_name, sep='\t',index=False, line_terminator='\n')
/var/folders/y0/1x_8r5js5_z6xvnv37zjwr6r0000gn/T/ipykernel_59613/1802922695.py:25: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  model_weights.to_csv(m_name, sep='\t',index=False, line_terminator='\n')
/var/folders/y0/1x_8r5js5_z6xvnv37zjwr6r0000gn/T/ipykernel_59613/1802922695.py:25: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  model_weights.to_csv(m_name, sep='\t',index=False, line_terminator='\n')


49 TRAIN: [   0    1    2 ... 3109 3110 3111] TEST: [   7   11   26   29   48   61   70   73   74   81  131  132  137  155
  159  170  175  187  236  248  252  302  309  313  315  328  370  377
  388  404  407  413  437  443  475  492  497  508  516  520  530  547
  555  558  570  578  586  589  590  593  605  612  613  618  632  633
  637  668  679  683  696  702  724  747  763  766  769  770  776  780
  800  806  808  811  812  815  824  843  848  857  859  867  869  877
  894  896  905  926  942  943  960  962  970  983  993  996 1013 1017
 1018 1032 1033 1068 1070 1081 1088 1091 1111 1114 1117 1132 1143 1149
 1155 1169 1173 1178 1186 1187 1191 1198 1203 1214 1216 1222 1253 1260
 1262 1289 1295 1299 1309 1313 1317 1318 1319 1324 1327 1347 1359 1361
 1368 1379 1380 1381 1385 1390 1391 1392 1405 1409 1428 1432 1451 1469
 1491 1497 1500 1503 1513 1516 1523 1525 1542 1543 1546 1553 1558 1564
 1567 1579 1583 1592 1595 1612 1620 1622 1623 1626 1670 1672 1673 1707
 1743 1744 1745 1773 1777

/var/folders/y0/1x_8r5js5_z6xvnv37zjwr6r0000gn/T/ipykernel_59613/1802922695.py:25: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  model_weights.to_csv(m_name, sep='\t',index=False, line_terminator='\n')
/var/folders/y0/1x_8r5js5_z6xvnv37zjwr6r0000gn/T/ipykernel_59613/1802922695.py:25: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  model_weights.to_csv(m_name, sep='\t',index=False, line_terminator='\n')
/var/folders/y0/1x_8r5js5_z6xvnv37zjwr6r0000gn/T/ipykernel_59613/1802922695.py:36: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  AUC_out.to_csv(f"data/{name}_AUCs.txt", sep='\t',index=False, line_terminator='\n')


## Males

### Logistic regression model

In [10]:
table1 = table[table.gender == "M"]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    989
1    989
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [11]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#               'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#               'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.01, 0.02],
              'max_iter': [75, 100, 150],
              'l1_ratio': [0.2, 0.1]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.01, 0.02], 'l1_ratio': [0.2, 0.1],
                         'max_iter': [75, 100, 150]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [12]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.6243225531401534

Non-zero coefficients: 37

Best estimator: LogisticRegression(C=0.01, l1_ratio=0.1, n_jobs=-1, penalty='elasticnet',
                   random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.01, 'l1_ratio': 0.1, 'max_iter': 100}
Best score: 0.5620659282996945



,SNP,Coefficient
Index,,
1,rs7520918,-0.10687
2,rs112270735,-0.077184
3,rs444618,-0.054004
4,rs13161496,-0.044109
5,rs3733349,-0.035941
6,rs3785891,-0.022054
7,rs494312,-0.015573
8,rs62190394,-0.008948
9,rs12995314,-0.00536


## Females

### Logistic regression model

In [13]:
table1 = table[table.gender == "F"]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    567
1    567
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [14]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#              'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#              'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.005, 0.01],
              'max_iter': [25, 50, 75],
              'l1_ratio': [0.1, 0.2]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.005, 0.01], 'l1_ratio': [0.1, 0.2],
                         'max_iter': [25, 50, 75]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [15]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.56756840825036

Non-zero coefficients: 6

Best estimator: LogisticRegression(C=0.005, l1_ratio=0.1, max_iter=50, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.005, 'l1_ratio': 0.1, 'max_iter': 50}
Best score: 0.5115738908675196



,SNP,Coefficient
Index,,
1,rs6871718,-0.026889
2,rs1949362,-0.023602
3,rs6964,0.005616
4,rs11248057,0.005966
5,rs3850379,0.008878
6,rs3806760,0.016905


## NN

### Logistic regression model

In [16]:
table1 = table[table.inv_genotype=="NN"]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    304
1    338
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [17]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#              'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#              'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.01, 0.02, 0.05],
              'max_iter': [10, 25],
              'l1_ratio': [0.5, 0.4, 0.3]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.01, 0.02, 0.05], 'l1_ratio': [0.5, 0.4, 0.3],
                         'max_iter': [10, 25]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [18]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.609418794767985

Non-zero coefficients: 9

Best estimator: LogisticRegression(C=0.02, l1_ratio=0.3, max_iter=10, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.02, 'l1_ratio': 0.3, 'max_iter': 10}
Best score: 0.5288828992007361



,SNP,Coefficient
Index,,
1,rs7826007,-0.013424
2,rs4897753,0.001234
3,rs3822023,0.005838
4,rs6842271,0.010758
5,rs12434297,0.012576
6,rs6964,0.014571
7,rs9985581,0.020144
8,rs3806760,0.031815
9,rs11248057,0.106701


## NI

### Logistic regression model

In [19]:
table1 = table[table.inv_genotype=="NI"]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    739
1    747
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [20]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#               'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#               'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.005, 0.01],
              'max_iter': [10, 25, 50],
              'l1_ratio': [0.2, 0.1]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.005, 0.01], 'l1_ratio': [0.2, 0.1],
                         'max_iter': [10, 25, 50]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [21]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.5796736789286148

Non-zero coefficients: 8

Best estimator: LogisticRegression(C=0.01, l1_ratio=0.2, max_iter=10, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.01, 'l1_ratio': 0.2, 'max_iter': 10}
Best score: 0.5327258606255274



,SNP,Coefficient
Index,,
1,rs7520918,-0.045971
2,rs1949362,-0.017297
3,rs6532190,0.008321
4,rs17016235,0.00924
5,rs6842271,0.023003
6,rs9985581,0.03514
7,rs7682766,0.036363
8,rs3806760,0.047802


## II

### Logistic regression model

In [22]:
table1 = table[table.inv_genotype=="II"]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    513
1    471
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [23]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#               'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#               'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.005, 0.01, 0.02],
              'max_iter': [50, 100, 200],
              'l1_ratio': [0.2, 0.1]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.005, 0.01, 0.02], 'l1_ratio': [0.2, 0.1],
                         'max_iter': [50, 100, 200]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [24]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.6603758748132421

Non-zero coefficients: 26

Best estimator: LogisticRegression(C=0.01, l1_ratio=0.1, n_jobs=-1, penalty='elasticnet',
                   random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.01, 'l1_ratio': 0.1, 'max_iter': 100}
Best score: 0.5940495196987687



,SNP,Coefficient
Index,,
1,rs34333,-0.057329
2,rs494312,-0.05673
3,rs3733349,-0.055571
4,rs6871718,-0.051304
5,rs444618,-0.032258
6,rs1736103,-0.031732
7,rs7535292,-0.026481
8,rs13161496,-0.016119
9,rs4331494,-0.004139


## NN Males

### Logistic regression model

In [25]:
table1 = table[(table.gender == "M") & (table.inv_genotype=="NN")]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    191
1    216
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [26]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#               'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#               'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.05, 0.1, 0.5],
              'max_iter': [10, 25],
              'l1_ratio': [0.7, 0.6, 0.5]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.05, 0.1, 0.5], 'l1_ratio': [0.7, 0.6, 0.5],
                         'max_iter': [10, 25]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [27]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.7224403723094822

Non-zero coefficients: 32

Best estimator: LogisticRegression(C=0.1, l1_ratio=0.6, max_iter=10, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.1, 'l1_ratio': 0.6, 'max_iter': 10}
Best score: 0.5580189109136477



,SNP,Coefficient
Index,,
1,rs35776335,-0.213769
2,rs13161496,-0.193039
3,rs12499663,-0.150666
4,rs444618,-0.124434
5,rs112270735,-0.083023
6,rs1269243287,-0.055458
7,rs3912643,-0.048141
8,rs11097297,-0.047875
9,rs2128786,-0.03413


## NI Males

### Logistic regression model

In [28]:
table1 = table[(table.gender == "M") & (table.inv_genotype=="NI")]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    480
1    477
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [29]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#               'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#               'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.005, 0.01],
              'max_iter': [10, 25, 50],
              'l1_ratio': [0.2, 0.1]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.005, 0.01], 'l1_ratio': [0.2, 0.1],
                         'max_iter': [10, 25, 50]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [30]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.5835888364779874

Non-zero coefficients: 4

Best estimator: LogisticRegression(C=0.005, l1_ratio=0.1, max_iter=25, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.005, 'l1_ratio': 0.1, 'max_iter': 25}
Best score: 0.5212733636229314



,SNP,Coefficient
Index,,
1,rs7520918,-0.011899
2,rs9985581,0.016555
3,rs6842271,0.016555
4,rs3806760,0.027075


## II Males

### Logistic regression model

In [31]:
table1 = table[(table.gender == "M") & (table.inv_genotype=="II")]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    318
1    296
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [32]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#               'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#               'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.01, 0.02],
              'max_iter': [50, 75, 100],
              'l1_ratio': [0.2, 0.1]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.01, 0.02], 'l1_ratio': [0.2, 0.1],
                         'max_iter': [50, 75, 100]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [33]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.6759093999660037

Non-zero coefficients: 18

Best estimator: LogisticRegression(C=0.01, l1_ratio=0.1, max_iter=75, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.01, 'l1_ratio': 0.1, 'max_iter': 75}
Best score: 0.6007288190582128



,SNP,Coefficient
Index,,
1,rs34333,-0.092096
2,rs3733349,-0.053093
3,rs3785891,-0.013793
4,rs8064765,-0.003056
5,rs62075803,-0.002828
6,rs4796663,-0.002828
7,rs4028634,0.001729
8,rs2410595,0.003947
9,rs764324,0.00623


## NN Females

### Logistic regression model

In [34]:
table1 = table[(table.gender == "F") & (table.inv_genotype=="NN")]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    113
1    122
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [35]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#               'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#               'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.1, 0.5, 1],
              'max_iter': [10, 25],
              'l1_ratio': [0.7, 0.6, 0.5]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.1, 0.5, 1], 'l1_ratio': [0.7, 0.6, 0.5],
                         'max_iter': [10, 25]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [36]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.8581894675758015

Non-zero coefficients: 112

Best estimator: LogisticRegression(C=0.5, l1_ratio=0.6, max_iter=10, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.5, 'l1_ratio': 0.6, 'max_iter': 10}
Best score: 0.5679924242424242



,SNP,Coefficient
Index,,
1,rs6871718,-0.426813
2,rs999361,-0.426221
3,rs7826007,-0.365302
4,rs3733349,-0.346928
5,rs78197677,-0.284285
6,rs7535292,-0.26294
7,rs4859611,-0.257429
8,rs1949362,-0.254682
9,rs11601088,-0.247447


## NI Females

### Logistic regression model

In [37]:
table1 = table[(table.gender == "F") & (table.inv_genotype=="NI")]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    259
1    270
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [38]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#              'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#              'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.005, 0.01],
              'max_iter': [10, 25],
              'l1_ratio': [1, 0.9]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.005, 0.01], 'l1_ratio': [1, 0.9],
                         'max_iter': [10, 25]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [39]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.5

Non-zero coefficients: 0

Best estimator: LogisticRegression(C=0.005, l1_ratio=1, max_iter=10, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.005, 'l1_ratio': 1, 'max_iter': 10}
Best score: 0.5



,SNP,Coefficient
Index,,


## II Females

### Logistic regression model

In [40]:
table1 = table[(table.gender == "F") & (table.inv_genotype=="II")]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    195
1    175
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [41]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#               'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#               'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [1, 10, 20],
              'max_iter': [1600, 3200],
              'l1_ratio': [1, 0.9]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [1, 10, 20], 'l1_ratio': [1, 0.9],
                         'max_iter': [1600, 3200]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [42]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.8847765567765569

Non-zero coefficients: 131

Best estimator: LogisticRegression(C=10, l1_ratio=1, max_iter=3200, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 10, 'l1_ratio': 1, 'max_iter': 3200}
Best score: 0.5429274165806673



,SNP,Coefficient
Index,,
1,rs138844738,-4.155261
2,rs115879964,-4.155261
3,rs536718528,-3.617586
4,rs115448944,-3.150895
5,rs71607338,-2.57904
6,rs17324625,-2.427067
7,rs2433733,-1.988805
8,rs6842271,-1.715214
9,rs62075803,-1.652955
